# Chapter 08: Advanced Money Management

📈Join our community: https://discord.gg/wXjNPAc5BH

📚Read our book: https://www.amazon.com/gp/product/B09HG18CYL 

🖥️Quantreo's YouTube channel: https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
mt5.initialize()

True

### Extract current positions 

In [2]:
mt5.positions_get()

(TradePosition(ticket=1650408542, time=1696434304, time_msc=1696434304764, time_update=1696434304, time_update_msc=1696434304764, type=1, magic=234001, identifier=1650408542, reason=3, volume=1.0, price_open=113900.0, sl=114405.0, tp=113405.0, price_current=114155.0, swap=0.0, profit=-51.0, symbol='WINV23', comment='python script op', external_id=''),
 TradePosition(ticket=1650412485, time=1696434542, time_msc=1696434542141, time_update=1696434542, time_update_msc=1696434542141, type=0, magic=234002, identifier=1650412485, reason=3, volume=1.0, price_open=5168.5, sl=5118.5, tp=5218.5, price_current=5167.5, swap=0.0, profit=-10.0, symbol='WDOX23', comment='python script op', external_id=''),
 TradePosition(ticket=1650430145, time=1696435761, time_msc=1696435761073, time_update=1696435761, time_update_msc=1696435761073, type=1, magic=234005, identifier=1650430145, reason=3, volume=1.0, price_open=46.72, sl=47.25, tp=46.25, price_current=46.79, swap=0.0, profit=-0.07, symbol='BBAS3F', com

In [3]:
def resume():
    """ Return the current positions. Position=0 --> Buy """    
    # Define the name of the columns that we will create
    columns = ["ticket", "position", "symbol", "volume", "magic", "profit", "price", "tp", "sl","trade_size"]

    # Go take the current open trades
    list_current = mt5.positions_get()

    # Create a empty dataframe
    summary = pd.DataFrame()

    # Loop to add each row in dataframe
    for element in list_current:
        element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                       element.profit, element.price_open, element.tp,
                                       element.sl, mt5.symbol_info(element.symbol).trade_contract_size],
                                      index=columns).transpose()
        summary = pd.concat((summary, element_pandas), axis=0)
    
    try:
        summary["profit %"] = summary.profit / (summary.price * summary.trade_size * summary.volume)
        summary = summary.reset_index(drop=True)
    except:
        pass
    return summary

In [4]:
resume()

,ticket,position,symbol,volume,magic,profit,price,tp,sl,trade_size,profit %
0,1650408542,1,WINV23,1.0,234001,-48.0,113900.0,113405.0,114405.0,1.0,-0.000421
1,1650412485,0,WDOX23,1.0,234002,-10.0,5168.5,5218.5,5118.5,1.0,-0.001935
2,1650430145,1,BBAS3F,1.0,234005,-0.06,46.72,46.25,47.25,1.0,-0.001284


### TRAILING STOP LOSS

In [5]:
# Create a dictionnary to contain the maxiumum price of each symbol 
max_price = dict()
min_price = dict()

# Infinite loop
while True:
    
    # Extract the current open positions
    summary = resume()
    
    # Verification: Is there any open position?
    if summary.shape[0] >0:
        for i in range(summary.shape[0]):
            
            # Extract information
            row = summary.iloc[i]
            symbol = row["symbol"]
            
            # Add the key if it is not in the keys list
            if symbol not in max_price.keys():
                max_price[symbol]=row["price"]
                
            if symbol not in min_price.keys():
                min_price[symbol]=row["price"]
                
                  
        

            """ CASE 1: Change dynamicly the stop loss for a BUY ORDER """
            # Trailing stop loss for a buy order
            if row["position"] == 0:
               
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]
                
                
                # If current price is greater than preivous max price --> new max price
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price
                    
                
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_max_price:
                    difference = from_sl_to_curent_price - from_sl_to_max_price

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": row["price"],
                    "sl": row["sl"] + difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                    information = mt5.order_send(request)
                    print(information)
                    
            
            """ CASE 2: Change dynamicly the stop loss for a SELL ORDER """
            # Trailing stop loss for a sell order
            if row["position"] == 1:
                
               
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = row["sl"] - current_price
                from_sl_to_min_price = row["sl"] - min_price[symbol]
                
                 # If current price is greater than preivous max price --> new max price
                if current_price < min_price[symbol]:
                    min_price[symbol] = current_price
                                
                    
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_min_price:
                    difference = from_sl_to_curent_price - from_sl_to_min_price 

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": row["price"],
                    "sl": row["sl"] - difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                
                    information = mt5.order_send(request)
                    print(information)
    print(max_price)
    # Avoid noise
    time.sleep(1)
    
    

{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}
{'WINV23': 113900.0, 'WDOX23': 5168.5, 'BBAS3F': 46.72}


KeyboardInterrupt: 

In [6]:
def trailing_stop_loss():
    global max_price, min_price, summary
    
    # Extract the current open positions
    summary = resume()
    
    # Verification: Is there any open position?
    if summary.shape[0] >0:
        for i in range(summary.shape[0]):
            
            # Extract information
            row = summary.iloc[i]
            symbol = row["symbol"]
            
        
                         
        

            """ CASE 1: Change dynamicly the stop loss for a BUY ORDER """
            # Trailing stop loss for a buy order
            if row["position"] == 0:
               
                if symbol not in max_price.keys():
                    max_price[symbol]=row["price"]
                
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]
                
                
                # If current price is greater than preivous max price --> new max price
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price
                    
                
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_max_price:
                    difference = from_sl_to_curent_price - from_sl_to_max_price

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": row["price"],
                    "sl": row["sl"] + difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                    information = mt5.order_send(request)
                    print(information)
                    
            
            """ CASE 2: Change dynamicly the stop loss for a SELL ORDER """
            # Trailing stop loss for a sell order
            if row["position"] == 1:
                
                if symbol not in min_price.keys():
                    min_price[symbol]=row["price"]
                    
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = row["sl"] - current_price
                from_sl_to_min_price = row["sl"] - min_price[symbol]
                
                 # If current price is greater than preivous max price --> new max price
                if current_price < min_price[symbol]:
                    min_price[symbol] = current_price
                                
                    
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_min_price:
                    difference = from_sl_to_curent_price - from_sl_to_min_price 

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": row["price"],
                    "sl": row["sl"] - difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                
                    information = mt5.order_send(request)
                    print(information)

In [7]:
def verif_tsl():
    global max_price, min_price, summary
    
    #print("MAX", max_price)
    
    #print("MIN", min_price)
    
    if len(summary)>0:
        buy_open_positions = summary.loc[summary["position"]==0]["symbol"]
        sell_open_positions = summary.loc[summary["position"]==1]["symbol"]
    else:
        buy_open_positions = []
        sell_open_positions = []
    
    """ IF YOU CLOSE ONE OF YOUR POSITION YOU NEED TO DELETE THE PRICE IN THE MAX AND MIN PRICES DICTIONNARIES"""
    if len(max_price) != len(buy_open_positions) and len(buy_open_positions) >0:
        symbol_to_delete = []
        
        for symbol in max_price.keys():

            if symbol not in list(buy_open_positions):
                symbol_to_delete.append(symbol)
        
        for symbol in symbol_to_delete:
            del max_price[symbol]
            
    if len(min_price) != len(sell_open_positions) and len(sell_open_positions) >0:
        symbol_to_delete = []
        
        for symbol in min_price.keys():

            if symbol not in list(sell_open_positions):
                symbol_to_delete.append(symbol)
        
        for symbol in symbol_to_delete:
            del min_price[symbol]
    
    if len(buy_open_positions) == 0:
        max_price={}
        
    if len(sell_open_positions) == 0:
        min_price={}
    

In [8]:
# Create a dictionnary to contain the maxiumum price of each symbol 
max_price = dict()
min_price = dict()

# Infinite loop
while True:
    trailing_stop_loss()

    
    verif_tsl()
    
    
    time.sleep(1)

KeyboardInterrupt: 

### Basket TP/SL

In [9]:
def verification_basket_sl(sl):
    """We need to put the sign of the sl:
    Stop loss -1%: sl=-1
    """
    list_current_position = list(mt5.positions_get())
    sum_price = sum_profit = 0
    for position in list_current_position:
        symbol = position.symbol
        trade_contract_size = mt5.symbol_info(symbol).trade_contract_size
        sum_price += (position.volume * position.price_open * trade_contract_size)
        sum_profit += position.profit
    profit_in_pct = 100 * sum_profit / sum_price
    condition = profit_in_pct < sl
    print(profit_in_pct)
    return condition

In [10]:
verification_basket_sl(-1)

-0.05381344214450512


False

In [ ]:
def verification_basket_tp(tp):
    list_current_position = list(mt5.positions_get())
    sum_price = sum_profit = 0
    for position in list_current_position:
        symbol = position.symbol
        trade_contract_size = mt5.symbol_info(symbol).trade_contract_size
        sum_price += (position.volume * position.price_open * trade_contract_size)
        sum_profit += position.profit
    profit_in_pct = 100 * sum_profit / sum_price
    condition = profit_in_pct > tp
    print(profit_in_pct)
    return condition